# 1. 1D Animated Plots

The idea is to generate animated plots and evtually include them in a [$\LaTeX$](https://www.latex-project.org/) file. To generate a simple 1D animated plot there are essentially two ways: 
1. Directly generate the whole animated plot witn Python and save it as a video fomrat, e.g. GIF or MP4; 
2. Generate a series of plots with python and save them as image format, e.g. PGN, PGF or SVG. 

The advantage of the first method is that it is a lot faster and probably easier to adjust. Moreover, obviously, you end up with a video that can be played in any reasonable video player. However, the major disadvantage is that I haven't found a good way to implement a GIF or MP4 video into LaTeX. There used to be a nice method using the package [media9](https://ctan.org/pkg/media9?lang=en) which relied on Flash. However since Adobe has ceased support of Flash, see e.g. [here](https://www.overleaf.com/learn/latex/Questions/How_can_I_embed_a_video_in_my_PDF_using_LaTeX%3F), you wont be able to play the video in Adobe Reader. It might still work if you open the file in another reader which supports Flash, I wouldn't know. 

The second method resolves this major drawback. The video will be played in Adobe Reader as desired as LaTeX creates the animation which does not rely on Flash. Apart from that, another major advantage of the second method is, that it allows you a high degree of freedom. E.g. it is possible to let LaTeX render the text insider your figure which will then be consistent with the rest of your text and, furthermore, also searchable. I'll elaborate that later in greater detail, see in Secion 1.4. For the time being, I'll start with explaining both methods for 1D plots.  

## 1.1. Create 1D Animated Plot in Video Format

In order to implement the first method, all python modules we need are [Numpy](https://numpy.org/) and [Matplotlib](https://matplotlib.org/). 

In [31]:
import numpy as np
import matplotlib.pyplot as plt
plt.ioff()      #stops jupyter to display figures if not explicitely called

As an example of this method, I'll draw the sine and cosine curve in the interval $[0, 10\pi]$. 

In [32]:
frames=500
x=np.linspace(0, 10*np.pi, frames)
fig, ax =plt.subplots()     # create empty figure

Now, we have to define a function which plots each frame of the plot. 

In [33]:
def animation(i): 
    xi=x[i:i+2]
    ax.plot(xi, np.sin(xi), color='r')
    ax.plot(xi, np.cos(xi), color='b')
    ax.set_xlim(0, 10*np.pi)        #displayed window of plot should be fixed
    ax.set_ylim(-1, 1)

    ax.set_title(r'$\sin(x),\cos(x),\ x \in [0, 2\pi]$')     #print title in LaTeX style

Be sure to pass raw strings or to escape the backslash. To actually animate this plot we use the class [FuncAnimation](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html) from Matplotlib. 

In [34]:
from matplotlib.animation import FuncAnimation

anim=FuncAnimation(fig, animation, frames=frames, interval=100, repeat=True)

anim.save(r'figures/sine_cosine_animated.gif')     #save as mp4 is also possible

Some important arguments to be specified are: *frames* determines how often the function animation is called, *interval* detrmines how long the pause between the next call of animation is in miliseconds and *repeat* which, if set to *True*, restarts the animation once it has finished for all eternity. If done correctly, the result should look like this: 
![Sine and cosine](sine_cosine_animated.gif)

## 1.2. Create 1D Animated Plots as Series of Images (Not Recommended)

This methods uses more basic and less fancy tools. We only create a certain number of pictures. This time the goal is to not to animate the sine and cosine curves, but rather dots moving on the since and cosine curves over the interval $[0, 10\pi]$, respectively. Since the output will be quite large, I'll print the moving dots not at every point on the curves.

In [35]:
frames=50
x1=np.linspace(0, 10*np.pi, 500)
x2=np.linspace(0, 10*np.pi, frames)
fig, ax =plt.subplots()

Now, we create our series of images. To avoid chaos (there're going to be 50 images) we save them in a seperate subfolder. Make sure that the name of each picture differs by some number, otherwise it will just overwrite the old one. 

In [37]:
for i in range(frames): 
    ax.clear()      #clear dots from step before
    ax.plot(x1, np.sin(x1), color='r')      #plot sine curve
    ax.plot(x1, np.cos(x1), color='b')      #plot cosine curve
    ax.set_title(r'$\sin(x),\ \cos(x),\ x \in [0, 2\pi]$') 
    ax.plot(x2[i], np.sin(x2[i]), marker='o', markerfacecolor='r', markeredgecolor='black', markersize=10)        #plot the dot
    ax.plot(x2[i], np.cos(x2[i]), marker='o', markerfacecolor='b', markeredgecolor='black', markersize=10)        #plot the dot
    fig.savefig(rf'figures/sine_cosine_png/im-{i}.png', dpi=500)       #names  of images are 'im-0.png' - 'im_49-png'

## 1.2. Animate png Images in LaTex 

To animate a series of pictures in LaTeX, we use the usepackage [animate](https://ctan.org/pkg/animate?lang=en). For demonstration, I will animate the the pictures of the moving dots on the sine and cosine cruves from before. An (almost) minimal code-example, saved as *sine_cosine_png.tex* looks like this: 

In the brackets of the command *\begin{animateinline}[]* you have to oppurtunity to pass general options like *loop* which ensures that the animation starts again once finished, *autoplay* which plays the animation as soon as you have reached the page on which the animation is located and *control* which adds an interface with buttons like play, fast forward, fast, backward, etc. In the first braces you can pass the frame rate, i.e. how many frames per second should be displayed. 

The first braces of the command *\multiframe* contain the total number of frames. In the second braces you specify a variable for looping trough the image names (I chose *i* in this case), followed by an $=$ sign and the starting point of the loop, a $+$ sign and the increment, i.e. the syntax of the second braces folow the scheme 
$$<variable>=<start>+<increment>. $$
The third braces contain a command for importing images, note that the numbers in the filename are replaced by *\i*, i.e. we call call the variable *i*. 

I recomment to open the generated PDF file in Adobe Acrobat Reader DC as you need a reader which supports animations. If done correctly, the PDF should look like this: 
[Sine and cosine](sine_cosine_png.pdf)

Let me draw the focus onto some drawbacks of this method: 
1. The pdf file gets really big (18 mb in my case), despite the fact that we only used 50 pictures in a, at best, mediocore resolution (only 500 dpi). This is in particular a problem if you want to send the file via e-mail where the limit for attachment files is often 5 mb; 
2. Though the title of the figure is written in LaTeX style, it is not consistent with the style of the rest of the text. Of course, it would be possible to manually align the text styles in either python or LaTeX, but this is neither pythoniac nor practical. Just imagine you hand in a perfectly manually aligned file to a journal for publishing and they use their own template for formatting and nothing will be aligned anymore; 
3. The text in the figure is not searchable (just type ctrl+f and search for 'sin' and their will only be one match instead of the desired two). 

Fortunately, all these dproblems can be rectified at one by simply using another format - the PGF format. 

## 1.4. The PGF Format

The [PGF format](https://en.wikipedia.org/wiki/Progressive_Graphics_File) makes it possible to let LaTeX directly render the text in your figures. So we go to the process of above before, but instead of saving the images as PNG, we save them as PGF. As the files get smaller, we can make more frames. 

In [40]:
frames=200
x1=np.linspace(0, 10*np.pi, 500)
x2=np.linspace(0, 10*np.pi, frames)

fig, ax =plt.subplots()
for i in range(frames): 
    ax.clear()      
    ax.plot(x1, np.sin(x1), color='r')
    ax.plot(x1, np.cos(x1), color='b')
    ax.set_title(r'$\sin(x),\ \cos(x),\ x \in [0, 2\pi]$') 
    ax.plot(x2[i], np.sin(x2[i]), marker='o', markerfacecolor='r', markeredgecolor='black', markersize=10)   
    ax.plot(x2[i], np.cos(x2[i]), marker='o', markerfacecolor='b', markeredgecolor='black', markersize=10)  
    #If you would like the labels of the ticks renderd in LaTeX too, you need to pass them in LaTeX form. 
    #Notice the empty string at the beginning and the end, this is necessesary you don't set ylim and xlim. 
    #I assume Matplotlib creates a thick at the beginning and the end which is not displayed. 
    ax.set_yticklabels(['', '$-1$', '$-0.75$', '$-0.5$', '$-0.25$', '$0$', '$0.25$', '$0.5$', '$0.75$', '$1$', ''])
    ax.set_xticklabels(['', '0', '5', '10', '15', '20', '25', '30',''])
    fig.savefig(rf'figures/sine_cosine_pgf/im-{i}.pgf')

C:\Users\User\AppData\Local\Temp\ipykernel_17760\157296426.py:16: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(['', '$-1$', '$-0.75$', '$-0.5$', '$-0.25$', '$0$', '$0.25$', '$0.5$', '$0.75$', '$1$', ''])
C:\Users\User\AppData\Local\Temp\ipykernel_17760\157296426.py:17: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['', '0', '5', '10', '15', '20', '25', '30',''])


## 1.5. Animate PGF Images in LaTeX

To import PGF files into LaTeX we need the LaTeX package [pgfplots](https://ctan.org/pkg/pgfplots?lang=en) to read PGF files. However, the LaTeX command *\\includegraphics* still does not accept PGF files. An alternative would be to use the LaTeX command *\\input*. However, I recommend using the package [import](https://ctan.org/pkg/import?lang=en) and the command *\\import*. In the particular case here, this is not necessary, though it will be in more complicated cases, for more on that see Chapter 2. An (almost) minimal LaTeX code-example looks as follows: 

If done correcrtly, the PDF should look like this: 
(Sine and cosine)[sine_cosine_pgf.pdf]

You can convince yourself that all the problems from above are resolved. Espeshially, the size of this PDF file is only a fraction from that one before - even though we quadrupled the frames. Moreover, unlike PNG, PGF also employs loosless data compression. Meaning that you can zoom into the image and you never see pixels. 

## 1.6. Use LaTeX Packages in Figures

If you play with the code from above, you'll notice that only commands from standard LaTeX work. As soon as you try to insert a code from, say, (amsmath)[https://ctan.org/pkg/amsmath?lang=en], you'll get an error. I'll find this particularly annoying since the command *\text* is from from amsmath. As soon as you try to write text in your title you must therefore write it putside the LaTeX command (outside the dollar signs *$*). But this has the consequence that LaTeX will not render this text and it will not align with the rest of the text in your PDF file. To overcome this issue, we have to tell python to use the amsart package. In this example I visualize the (complex) roots of the Polynomial 
$$ P_\gamma(z):=z^2-4z+4\gamma, \quad z \in \mathbb{C}, $$
where $\gamma$ is some parameter within the interval $[0, 2]$. 

In [41]:
import matplotlib as mpl

mpl.use('pgf')

pgf_latex = {                      #setup parameters of Matplotlib to use LaTeX for output
    "pgf.texsystem": "pdflatex",        
    "text.usetex": True,                #LaTeX will write all text
    "font.family": "serif",
    "font.serif": [],                    
    "font.sans-serif": [],              
    "font.monospace": [],
    "axes.labelsize": 10,               
    "font.size": 10,
    "legend.fontsize": 8,               
    "xtick.labelsize": 8,               
    "ytick.labelsize": 8,
    "pgf.preamble": "\n".join([         #plots will use this preamble
        r"\usepackage{amsmath}",
        ])
    }
mpl.rcParams.update(pgf_latex)

Of course, the roots of $P_\gamma$ are 
$$b\pm\sqrt{4-4\gamma}$$
if 16-16\gamma\geq 0$ and 
$$b\pm\mathrm{i}\sqrt{4\gamma-4}$$
otherwise. 

In [42]:
gamma=np.linspace(0, 2, 200)
fig, ax =plt.subplots()
for i in range(len(gamma)): 
    ax.clear()      #clear dots from step before
    ax.set_xlim(0, 4)
    ax.set_ylim(-2, 2)
    if 16-16*gamma[i]>=0: 
        x1=.5*(4+np.sqrt(16-16*gamma[i]))
        x2=.5*(4-np.sqrt(16-16*gamma[i]))
        ax.plot(x1, 0, marker='o', color='blue', markersize=20)
        ax.plot(x2, 0, marker='o', color='red', markersize=20)
    else: 
        y=.5*np.sqrt(16*gamma[i]-16)
        ax.plot(2, y, marker='o', color='blue', markersize=20)
        ax.plot(2, -y, marker='o', color='red', markersize=20)
    ax.set_title(rf'$\text{{Roots of}}\ P_\gamma(z):=z^2-4z+4\gamma$\ \text{{for}}\ $\gamma={gamma[i]:.2f}') 
    #If you would like the labels of the ticks renderd in LaTeX too, you need to pass them in LaTeX form. 
    #Notice the empty string at the beginning and the end, this is necessesary you don't set ylim and xlim. 
    #I assume Matplotlib creates a thick at the beginning and the end that is not displayed. 
    ax.set_yticklabels(['$-4$', '$-3$', '$-2$', '$-1',  '$0$', '$1$', '$2$', '$3$', '$4$'])
    ax.set_xticklabels(['$0$', '$0.5$', '$1$', '$1.5$', '$2$', '$2.5$', '$3$', '$3.5$', '$4$'])
    fig.savefig(rf'figures/roots/im-{i}.pgf')

C:\Users\User\AppData\Local\Temp\ipykernel_17760\2683981080.py:20: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(['$-4$', '$-3$', '$-2$', '$-1',  '$0$', '$1$', '$2$', '$3$', '$4$'])
C:\Users\User\AppData\Local\Temp\ipykernel_17760\2683981080.py:21: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['$0$', '$0.5$', '$1$', '$1.5$', '$2$', '$2.5$', '$3$', '$3.5$', '$4$'])


The LaTeX code for implementing this animated graphic is similar to the one before: 

You can convince yourself that the text in the title of the figure is aligned with the rest of the text in the PDF file. 
(Roots of $z^2-4z+4\gamma$)[roots.pdf]

<embed src="/roots.pdf" type="application/pdf">